# Load Processed Data into Vector Database

This notebook loads output from data prep kit into Milvus

**Step-4 in this workflow**

![](../media/rag-overview-2.png)


## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Load Parquet Data

Load all  `.parquet` files in the given dir

In [2]:
import pandas as pd
import glob

print ('Loading data from : ', MY_CONFIG.OUTPUT_FOLDER_FINAL)

# Get a list of all Parquet files in the directory
parquet_files = glob.glob(f'{MY_CONFIG.OUTPUT_FOLDER_FINAL}/*.parquet')
print ("Number of parquet files to read : ", len(parquet_files))
print ()

# Create an empty list to store the DataFrames
dfs = []

# Loop through each Parquet file and read it into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    print (f"Read file: '{file}'.  number of rows = {df.shape[0]}")
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
data_df = pd.concat(dfs, ignore_index=True)

print (f"\nTotal number of rows = {data_df.shape[0]}")

Loading data from :  output/output_final
Number of parquet files to read :  2

Read file: 'output/output_final/granite.parquet'.  number of rows = 123
Read file: 'output/output_final/attension.parquet'.  number of rows = 88

Total number of rows = 211


In [3]:

## Shape the data

MY_CONFIG.EMBEDDING_LENGTH =  len(data_df.iloc[0]['embeddings'])
print ('embedding length: ', MY_CONFIG.EMBEDDING_LENGTH)

# rename 'embeddings' columns as 'vector' to match default schema
# if 'vector' not in data_df.columns and 'embeddings' in data_df.columns:
#     data_df = data_df.rename( columns= {'embeddings' : 'vector'})
# if 'text' not in data_df.columns and 'contents' in data_df.columns:
#     data_df = data_df.rename( columns= {'contents' : 'text'})

data_df = data_df.rename( columns= {'embeddings' : 'vector', 'contents' : 'text'})

print (data_df.info())
data_df.head(3)

embedding length:  384
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   filename            211 non-null    object 
 1   num_pages           211 non-null    int64  
 2   num_tables          211 non-null    int64  
 3   num_doc_elements    211 non-null    int64  
 4   ext                 211 non-null    object 
 5   hash                211 non-null    object 
 6   size                211 non-null    int64  
 7   date_acquired       211 non-null    object 
 8   pdf_convert_time    211 non-null    float64
 9   source_filename     211 non-null    object 
 10  source_document_id  211 non-null    object 
 11  text                211 non-null    object 
 12  doc_jsonpath        211 non-null    object 
 13  page_number         211 non-null    int64  
 14  bbox                211 non-null    object 
 15  document_id         211 non-null  

,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,text,doc_jsonpath,page_number,bbox,document_id,chunk_id,removed,chunk_hash,vector
0,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,Granite Code Models: A Family of Open Foundati...,$.main-text[3],1,"[142.70646667, 672.96929932, 468.58251953, 711...",b773445f7cf4cc9a5bf6ec296c74504f93c9c179028ac6...,88,[],-1,"[-0.015789315, -0.07841933, -0.032271657, 0.00..."
1,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,Granite Code Models: A Family of Open Foundati...,$.main-text[4],1,"[107.61845398, 535.62896729, 503.99923706, 647...",7353bcc8d99c279335eaf120c793ca6a08f9a4fddcbb5b...,89,[],-1,"[-0.059480786, -0.056680508, -0.042864937, -0...."
2,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,Granite Code Models: A Family of Open Foundati...,$.main-text[5],1,"[220.87228394, 484.46414185, 390.87872314, 529...",389267895ca214924a0a071df8379c2b15fcf374f232a6...,90,[],-1,"[-0.07557265, -0.07152908, -0.048923455, -0.04..."


## Step-3: Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>




In [4]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)

print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)

✅ Connected to Milvus instance: ./rag_1_dpk.db


# Step-4: Create A Collection



In [5]:
# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name=MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name=MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


milvus_client.create_collection(
    collection_name=MY_CONFIG.COLLECTION_NAME,
    dimension=MY_CONFIG.EMBEDDING_LENGTH,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    auto_id=True
)
print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)


✅ Created collection : dpk_papers


In [6]:
res = milvus_client.insert(collection_name=MY_CONFIG.COLLECTION_NAME, data=data_df.to_dict('records'))

print('inserted # rows', res['insert_count'])

milvus_client.get_collection_stats(MY_CONFIG.COLLECTION_NAME)

inserted # rows 211


{'row_count': 211}

## Step-5: Close DB Connection

Close the connection so the lock files are relinquished and other notebooks can access the db

In [7]:
milvus_client.close()

print ("✅ SUCCESS")


✅ SUCCESS


## Test your data by doing a Vector Search

See notebook [vector_search.ipynb](vector_search.ipynb)